In [74]:
import pandas as pd
import plotly.express as px
import geopandas as gpd
import folium

import numpy as np

In [131]:
# Load data for the number of wind turbines and total energy produced
data = pd.read_csv('cleaned_data.csv')

In [132]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      760 non-null    int64  
 1   ref_id                          760 non-null    int64  
 2   last_updated                    758 non-null    object 
 3   operator_or_applicant           743 non-null    object 
 4   site_name                       760 non-null    object 
 5   capacity_mw                     760 non-null    float64
 6   turbine_capacity_mw             759 non-null    float64
 7   no_of_turbines                  760 non-null    float64
 8   height_of_turbines_m            73 non-null     float64
 9   development_status              760 non-null    object 
 10  development_status_short        760 non-null    object 
 11  address                         759 non-null    object 
 12  county                          760 

In [133]:
# Convert the date column to a proper datetime format
data['last_updated'] = pd.to_datetime(data['last_updated'])

In [134]:
data['decom_year'] = data['decom_year'].astype('Int64')
data['start_year'] = data['start_year'].astype('Int64')

In [135]:
data

,Unnamed: 0,ref_id,last_updated,operator_or_applicant,site_name,capacity_mw,turbine_capacity_mw,no_of_turbines,height_of_turbines_m,development_status,...,appeal_lodged,appeal_withdrawn,appeal_refused,appeal_granted,planning_permission_granted,planning_permission_expired,under_construction,operational,decom_year,start_year
0,240,2948,2015-01-09,Wind Electric,Delabole Wind Farm,4.0,0.40,10.0,NaN,Decommissioned,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1991-11-01,2015,1991
1,216,2921,2009-07-06,Yorkshire Water,Chelker Reservoir,1.3,0.30,4.0,NaN,Operational,...,NaN,NaN,NaN,NaN,1991-07-12,NaN,NaN,1992-06-01,<NA>,1992
2,284,3005,2015-01-09,Scottish Power Renewables,Carland Cross,6.0,0.40,15.0,NaN,Decommissioned,...,NaN,NaN,NaN,NaN,1992-04-29,NaN,NaN,1992-08-27,2015,1992
3,823,3659,2013-10-08,Border Wind Farms Ltd,Blyth Harbour Wind Farm,2.7,0.30,9.0,NaN,Operational,...,NaN,NaN,NaN,NaN,1992-03-06,NaN,NaN,1992-12-01,<NA>,1992
4,107,2713,2019-08-05,Npower Renewables,Kirkby Moor,4.8,0.40,12.0,NaN,Operational,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1993-01-01,<NA>,1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2105,7562,2020-09-23,Statkraft UK,Windy Rig Wind Farm,43.0,3.00,12.0,125.0,Operational,...,NaN,NaN,NaN,NaN,2019-11-08,2022-11-07,2020-03-01,2022-01-10,<NA>,2022
756,1981,6422,2023-03-16,Cleanearth,Graig Fatha Farm,2.5,2.50,1.0,126.0,Operational,...,NaN,NaN,NaN,NaN,2018-03-14,2023-03-14,2021-10-01,2022-02-01,<NA>,2022
757,2054,7304,NaT,Green Cat Renewables,Deuchries Windfarm (extension),2.0,1.00,2.0,119.0,Operational,...,NaN,NaN,NaN,NaN,2019-10-08,2022-10-08,NaN,2022-04-01,<NA>,2022
758,1424,4300,2023-01-16,ERG,Creag Riabhach Wind Farm,90.0,4.20,22.0,125.0,Operational,...,NaN,NaN,NaN,NaN,2016-10-17,NaN,2021-05-24,2023-01-12,<NA>,2023


In [136]:
def date_converter(dataframe, column_list, fmt):
    for column in column_list:
        dataframe[column] = pd.to_datetime(dataframe[column], format=fmt)
    return data

In [137]:
data = date_converter(data, ['decom_year','start_year'], fmt='%Y')
data = date_converter(data, ['last_updated','operational','planning_permission_granted'], fmt='%Y-%m-%d')

In [138]:
data

,Unnamed: 0,ref_id,last_updated,operator_or_applicant,site_name,capacity_mw,turbine_capacity_mw,no_of_turbines,height_of_turbines_m,development_status,...,appeal_lodged,appeal_withdrawn,appeal_refused,appeal_granted,planning_permission_granted,planning_permission_expired,under_construction,operational,decom_year,start_year
0,240,2948,2015-01-09,Wind Electric,Delabole Wind Farm,4.0,0.40,10.0,NaN,Decommissioned,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,1991-11-01,2015-01-01,1991-01-01
1,216,2921,2009-07-06,Yorkshire Water,Chelker Reservoir,1.3,0.30,4.0,NaN,Operational,...,NaN,NaN,NaN,NaN,1991-07-12,NaN,NaN,1992-06-01,NaT,1992-01-01
2,284,3005,2015-01-09,Scottish Power Renewables,Carland Cross,6.0,0.40,15.0,NaN,Decommissioned,...,NaN,NaN,NaN,NaN,1992-04-29,NaN,NaN,1992-08-27,2015-01-01,1992-01-01
3,823,3659,2013-10-08,Border Wind Farms Ltd,Blyth Harbour Wind Farm,2.7,0.30,9.0,NaN,Operational,...,NaN,NaN,NaN,NaN,1992-03-06,NaN,NaN,1992-12-01,NaT,1992-01-01
4,107,2713,2019-08-05,Npower Renewables,Kirkby Moor,4.8,0.40,12.0,NaN,Operational,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,1993-01-01,NaT,1993-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2105,7562,2020-09-23,Statkraft UK,Windy Rig Wind Farm,43.0,3.00,12.0,125.0,Operational,...,NaN,NaN,NaN,NaN,2019-11-08,2022-11-07,2020-03-01,2022-01-10,NaT,2022-01-01
756,1981,6422,2023-03-16,Cleanearth,Graig Fatha Farm,2.5,2.50,1.0,126.0,Operational,...,NaN,NaN,NaN,NaN,2018-03-14,2023-03-14,2021-10-01,2022-02-01,NaT,2022-01-01
757,2054,7304,NaT,Green Cat Renewables,Deuchries Windfarm (extension),2.0,1.00,2.0,119.0,Operational,...,NaN,NaN,NaN,NaN,2019-10-08,2022-10-08,NaN,2022-04-01,NaT,2022-01-01
758,1424,4300,2023-01-16,ERG,Creag Riabhach Wind Farm,90.0,4.20,22.0,125.0,Operational,...,NaN,NaN,NaN,NaN,2016-10-17,NaN,2021-05-24,2023-01-12,NaT,2023-01-01


In [139]:
#Sum the number of turbines annually
years_turbines = data.groupby('start_year')['no_of_turbines'].sum()
years_turbines = pd.DataFrame(years_turbines).reset_index()
years_turbines

,start_year,no_of_turbines
0,1991-01-01,10.0
1,1992-01-01,28.0
2,1993-01-01,209.0
3,1994-01-01,53.0
4,1995-01-01,56.0
5,1996-01-01,122.0
6,1997-01-01,88.0
7,1998-01-01,38.0
8,1999-01-01,34.0
9,2000-01-01,90.0


In [140]:
#Sum the number of projects decommissioned each year
decom_projects = data[data['decom_year'] != np.NaN]
decom_projects = decom_projects.groupby('decom_year')['no_of_turbines'].sum()
decom_projects

decom_year
2015-01-01    76.0
2019-01-01     2.0
Name: no_of_turbines, dtype: float64

In [141]:
#Bring the two together to find out the number of projects active each year
total_turbs = pd.merge(years_turbines, decom_projects, how='left', left_on='start_year', right_on='decom_year')
total_turbs.rename(columns={'no_of_turbines_x':'operational','no_of_turbines_y':'decommissioned'}, inplace=True)
total_turbs

,start_year,operational,decommissioned
0,1991-01-01,10.0,NaN
1,1992-01-01,28.0,NaN
2,1993-01-01,209.0,NaN
3,1994-01-01,53.0,NaN
4,1995-01-01,56.0,NaN
5,1996-01-01,122.0,NaN
6,1997-01-01,88.0,NaN
7,1998-01-01,38.0,NaN
8,1999-01-01,34.0,NaN
9,2000-01-01,90.0,NaN


In [142]:
total_turbs['decommissioned'] = total_turbs['decommissioned'].fillna(0)
total_turbs['total'] = total_turbs['operational'] - total_turbs['decommissioned']
total_turbs['cumul_total'] = total_turbs['total'].cumsum()
total_turbs

,start_year,operational,decommissioned,total,cumul_total
0,1991-01-01,10.0,0.0,10.0,10.0
1,1992-01-01,28.0,0.0,28.0,38.0
2,1993-01-01,209.0,0.0,209.0,247.0
3,1994-01-01,53.0,0.0,53.0,300.0
4,1995-01-01,56.0,0.0,56.0,356.0
5,1996-01-01,122.0,0.0,122.0,478.0
6,1997-01-01,88.0,0.0,88.0,566.0
7,1998-01-01,38.0,0.0,38.0,604.0
8,1999-01-01,34.0,0.0,34.0,638.0
9,2000-01-01,90.0,0.0,90.0,728.0


In [143]:
#Did a similar thing to the above already elsewhere to calculate the energy produced annually
years_mw = pd.read_csv('years_mw.csv')
years_mw

,Unnamed: 0,start_year,ops_capacity,decom_capacity,total,cumulative_capacity,load_factor,GWh_annual,TWh_annual
0,0,1991,4.0,0.0,4.0,4.0,0.27,9.46080,0.009461
1,1,1992,10.0,0.0,10.0,14.0,0.27,33.11280,0.033113
2,2,1993,75.1,0.0,75.1,89.1,0.27,210.73932,0.210739
3,3,1994,25.4,0.0,25.4,114.5,0.27,270.81540,0.270815
4,4,1995,30.6,0.0,30.6,145.1,0.27,343.19052,0.343191
5,5,1996,70.4,0.0,70.4,215.5,0.27,509.70060,0.509701
6,6,1997,59.3,0.0,59.3,274.8,0.27,649.95696,0.649957
7,7,1998,17.1,0.0,17.1,291.9,0.31,792.68364,0.792684
8,8,1999,20.9,0.0,20.9,312.8,0.28,767.23584,0.767236
9,9,2000,56.4,0.0,56.4,369.2,0.27,873.23184,0.873232


In [144]:
# Create a line plot for the number of wind turbines over time
fig1 = px.line(total_turbs, x='start_year', y='cumul_total', title='Number of Onshore Wind Turbines in the UK')

# Create a line plot for the total energy produced over time
fig2 = px.line(years_mw, x='start_year', y='GWh_annual', title='Total Energy Produced by Onshore Wind Turbines (GWh)')

In [189]:
#Need to convert my coordinates from British National Grid to Lat/Long to work with Folium map

import pyproj
import geopandas as gpd

# Define the BNG coordinate system (EPSG:27700)
bng = pyproj.CRS("EPSG:27700")

# Define the WGS84 coordinate system (EPSG:4326)
wgs84 = pyproj.CRS("EPSG:4326")

# Create a PyProj Transformer for the coordinate conversion
transformer = pyproj.Transformer.from_crs(bng, wgs84, always_xy=True)

# Convert BNG coordinates to latitude and longitude
converted_coords = transformer.transform(data['xcoord'].values, data['ycoord'].values)
data['longitude'], data['latitude'] = converted_coords[0], converted_coords[1]

# Print the converted coordinates
print(data[['xcoord', 'ycoord', 'latitude', 'longitude']])

       xcoord    ycoord   latitude  longitude
0    208620.0   85190.0  50.634907  -4.707688
1    405730.0  451820.0  53.962432  -1.914154
2    184500.0   54500.0  50.350880  -5.030587
3    432290.0  581345.0  55.125438  -1.495191
4    326111.0  485242.0  54.257491  -3.135725
..        ...       ...        ...        ...
755  261700.0  600000.0  55.274673  -4.178561
756  301279.0  185404.0  51.558849  -3.425452
757  363721.0  853713.0  57.572018  -2.608187
758  252589.0  928364.0  58.219301  -4.511210
759  144149.0  490451.0  54.245607  -5.928678

[760 rows x 4 columns]


In [194]:
# Create a map showing the wind turbine locations
uk_map = folium.Map(location=[54.7023545, -3.2765753], zoom_start=6)

#Loop through each row in the dataframe
for i,row in data.iterrows():
    #Setup the content of the popup
    iframe = folium.IFrame('Site:' + str(row['site_name']))
    
    #Initialise the popup using the iframe
    popup = folium.Popup(iframe, min_width=200, max_width=300)
    
    #Add each row to the map
    marker = folium.Marker(location=[row['latitude'],row['longitude']],
                  popup = popup)
    marker.add_to(uk_map)
    
uk_map.save("map.html")
uk_map

In [195]:
# Combine the figures into a dashboard layout (adjust the size and arrangement as needed)
fig1.update_layout(height=400)
fig2.update_layout(height=400)
uk_map = uk_map._repr_html_()

In [197]:
# Save the HTML file or launch a web server to view the dashboard
dashboard_html = '''
<!DOCTYPE html>
<html>
<head>
    <title>Onshore Wind Turbines Dashboard</title>
</head>
<body>
    <h1>Number of Onshore Wind Turbines in the UK</h1>
    {}
    <h1>Total Energy Produced by Onshore Wind Turbines</h1>
    {}
    <h1>Wind Turbine Locations</h1>
    {}
</body>
</html>
'''.format(fig1.to_html(full_html=False), fig2.to_html(full_html=False), uk_map)

with open('wind_turbines_dashboard.html', 'w') as f:
    f.write(dashboard_html)